In [12]:
import scipy.cluster.hierarchy as sphier
import scipy.spatial.distance as spdist
import pymisca.vis_util as pyvis
import pymisca.util as pyutil
plt = pyvis.plt

import numpy as np

is in ipython: 1
is in ipython: 1
[WARN] pymisca.vis_util cannot find network


In [18]:
spdist.squareform(np.ojobnes((3,3)),checks=0)

array([1., 1., 1.])

In [47]:
import scipy.cluster.hierarchy as sphier
import scipy.spatial.distance as spdist
import pymisca.vis_util as pyvis
import pymisca.util as pyutil
plt = pyvis.plt

# sphier.dendrogram??

In [ ]:
# cat ~/bash.he

In [65]:
%%writefile __init__.py
import CountMatrix as scount

Overwriting __init__.py


In [1]:
%%writefile norm.py
import numpy as np
import pymisca.util as pyutil

import sklearn.decomposition as skdecomp
pd = pyutil.pd
import CountMatrix as scount
# import util as util
import sys
import pymisca.ext
snorm = sys.modules[__name__]


def stdNorm(X,meanNorm=0,
            normF = pyutil.l2norm):
    deco = 0
    if isinstance(X, scount.countMatrix):
        deco =1
        df = X 
    if meanNorm:
        X = snorm.meanNorm(X)
    if isinstance(X, pd.DataFrame):
        X = X.values
    X = X.copy()
    assert isinstance(X,np.ndarray)
    
    STD = normF(X,axis=1,keepdims=1); pos = np.squeeze(STD>0);
    X[pos] = X[pos]/STD[pos]

    if deco:
        X = df.setDF(X)
        X.param['normF'] = 'stdNorm'
    return X

def meanNorm(X,axis=1):
    deco = 0
    if isinstance(X,scount.countMatrix):
        deco =1
        df = X 
        
    if isinstance(X,pd.Series):
        X = X.values
        X = (X - X.mean())
    else:
        if isinstance(X, pd.DataFrame):
            X = X.values
            
        assert isinstance(X,np.ndarray)
        X = (X-X.mean(axis=axis,keepdims=1))
        
    param = getattr(X,'param',{})    
    param['normF'] = 'meanNorm'
#     X.param = param
    
    if deco:
#         cols = X.columns
        X = df.setDF(X)
#         X.columns = cols
        X.param['normF'] = 'meanNorm'        
    return X

def sumNorm(X,axis=1):
    deco = 0
    if isinstance(X,scount.countMatrix):
        deco =1
        df = X 
        
    if isinstance(X,pd.Series):
        X = X.values
        SUM = X.sum()
        if SUM != 0:
            X = (X / SUM)
    else:
        if isinstance(X, pd.DataFrame):
            X = X.values
            
        assert isinstance(X,np.ndarray)
        SUM =X.sum(axis=axis,keepdims=1)
        SUM[SUM==0] = 1.
        X = (X / SUM)
        
    if deco:
#         cols = X.columns
        X = df.setDF(X)
#         X.columns = cols
        X.param['normF'] = 'sumNorm'        
    return X

def diffNorm(X,axis=1):
    deco = 0
    if isinstance(X,scount.countMatrix):
        deco =1
        df = X 
        
    if isinstance(X,pd.Series):        
        X = X.values
        X = np.diff(X,axis=0)
    else:
        if isinstance(X, pd.DataFrame):
            X = X.values            
        assert isinstance(X,np.ndarray)
        X = np.diff(X,axis=axis)
#         X = (X-X.mean(axis=axis,keepdims=1))
        
    param = getattr(X,'param',{})    
    param['normF'] = 'diffNorm'
#     X.param = param
    
    if deco:
#         cols = X.columns
        X = df.setDF(X)
#         X.columns = cols
        X.param['normF'] = 'diffNorm'        
    return X

def meanNormProj(X):
    deco = 0
    if isinstance(X,scount.countMatrix):
        deco =1
        df = X
        
    X = meanNorm(X)
    if isinstance(X, pd.DataFrame):
        X = X.values    
        
    D = X.shape[1]
    Wn  = pyutil.meanNormBasis(D,orthonormal =1 )
    X = X.dot(Wn.T)
    
    param = getattr(X,'param',{})    
    param['normF'] = 'meanNormProj'
#     X.param = param

    if deco:
        X = df.setDF(X)
        X.param['normF'] = 'meanNormProj'        
    return X
import sklearn.decomposition as skdecomp
def meanNormPCA(X,cutoff=1.0, withModel=0):
    name = 'meanNormPCA'
    deco = 0
    if isinstance(X,scount.countMatrix):
        deco =1
        df = X
        
    X = meanNorm(X)
    X = X.values if isinstance(X, pd.DataFrame) else X
    
    X0 = X
    mdl = skdecomp.PCA()
    mdl.fit(X)
    
    X = mdl.transform(X)
    
    first = np.cumsum(mdl.explained_variance_ratio_) >= cutoff
    idx   =np.argmax(first) + 1
    mdl.components_ = mdl.components_[:idx]
#     mdl.mean_ = mdl.mean_[:,idx]

    X = X[:,:idx]
        
    param = getattr(X,'param',{})    
    param['normF'] = name
#     X.param = param

    if deco:
        X = df.setDF(X)
        X.param['normF'] = name     
    if withModel:
        return (X,mdl,X0)
    else:
        return X
def ctNorm(X):
    X = (X-X.mean(axis=1,keepdims=1))
    return X
def identityNorm(X):
    param = getattr(X,'param',{})    
    param['normF'] = 'identityNorm'
#     X.param = param    
    return X



Overwriting norm.py


In [17]:
%%writefile qcmsg.py

def msgGMM(model = None, train_data = None,name='test',**kwargs):
    mdl = model
    s = '''
Name:{name}
Converged:{cvg}
min_logL: {logL}
(lower-bound of) MEAN logL :{mlogL}'''.format(
        name=name,
         cvg=mdl.converged_,
         logL = mdl.lower_bound_,
         mlogL=mdl.lower_bound_/len(train_data) if not train_data is None else mdl.lower_bound_
                                )
    return s

Overwriting qcmsg.py


In [90]:
%%writefile modelRoutine.py

import sklearn.mixture as skmix
import sklearn.decomposition.pca as skpca

import pymisca.util as pyutil
np = pyutil.np; pd = pyutil.pd
import os
import qcmsg
import synotil.CountMatrix as scount
# import synotil.qcplots as qcplots

def fit_PCA(C,n_components=5,**kwargs):
    mdl = skpca.PCA(n_components=n_components,**kwargs)
    M = mdl.fit_transform(C)
    
    resDict = {'model':mdl,
            'train_data':C,
            'trans_data':M,}

    return pyutil.util_obj(**resDict)


def fit_BGM(C,
            ALI = 'Test',
#             DIR = ''
#             normF = identityNorm,
            stdPer = 0,
            rowName=None,
            colName=None,
            nClu = 25,
            maxIt = 1000,
            algo = 'DPGMM',
            DIR= '.',
#            algoLst = ['DPGMM'],
            alpha = .1,
            covariance_type = 'diag',
            fixMean = 0,
            reorder=1,
            model_only =0,
            random_state= None,
            dbg=0,
#             covariance_type = None,
#             **kwargs
           ):
    '''
Fit an BayesianGaussianMixture() model from sklearn
'''
#     if algoLst is None:
#         algoLst = ['DPGMM','DDGMM','GMM',]    
    try:
        DIR,ALI = ALI.rsplit('/',1)
    except:
        DIR= DIR
    os.system('mkdir -p %s'%(DIR))
    
    
    ###### Manage meta attributes of the model ########
    param = {
            'fixMean':fixMean,
             'stdPer':stdPer,
             'nClu':nClu,
             'genre':algo,
             'covarianceType': covariance_type,
              'maxIt' : maxIt,
              'randomState':random_state,
         }
    param.update(getattr(C,'param',{}))
    
    ####### Convert to numpy arrary ######
    if isinstance(C,pd.DataFrame):
        if  ALI=='Test':
            ALI = getattr(C,'name','Test') 

        rowName,colName,C = C.index.values, C.columns, C.values
        pass
    
    ##### Old routine that filter by STD ###########
    if stdPer > 0 :
        assert stdPer < 100,'Percentile must < 100, Got %d instead'%stdPer
        (MEAN,STD,CV),_ = qc_Avg(C)
        pIdx = STD > np.percentile(STD, stdPer)        
        rowName = np.array(rowName)[pIdx]; C = C[pIdx]
    print '[ALI]=',ALI
    nFeat = C.shape[-1]
        
    #####====== Defnitions of fitters=========#######
    
    ###### Arguments shared among fitters ######
    common = {'n_components': nClu,
          'verbose':2,
         'max_iter':maxIt,
             'covariance_type':covariance_type,
              'random_state':random_state,
             }
    if fixMean:
        mean_precision_prior = 1E-128
        mean_prior = [0.]*nFeat
    else:
        mean_precision_prior  = None
        mean_prior = None
        
    ####### List of fitters ######
    mdlLst = {'DPGMM':skmix.BayesianGaussianMixture(weight_concentration_prior_type='dirichlet_process',
                                        weight_concentration_prior=alpha,
                                        mean_precision_prior = mean_precision_prior,
                                        mean_prior = mean_prior,
                                       **common),
          'GMM':skmix.GaussianMixture(**common),
          'DDGMM':skmix.BayesianGaussianMixture(weight_concentration_prior_type='dirichlet_distribution',
                                        weight_concentration_prior=alpha,
                                        mean_precision_prior = mean_precision_prior,
                                        mean_prior = mean_prior,
                                       **common),
         }
    
    
    ############# Select model by "algo"####
    X = C
#     if dbg >= 2:
#         qcplots.qc_Avg(C,silent=0)
    print pyutil.qc_matrix(X)
    mdl = mdlLst.get(algo,None)
    assert mdl is not None, 'Algorithm %s not found '%algo
    
    NAME = '%s_%s'%(ALI,pyutil.dict2flat(param))    
    print '[MSG] Now Fitting Model:%s'%NAME
    
    
    
    ####### Meta data of the training Data #######
    d = {'name': NAME,
         'train_data':X,
         'colName':colName,
         'rowName':rowName,
         'param':param,
       }
    
    
    ##### Fitting model and caching the result to specified DIR/NAME ####
    try:
        logFile = open('%s/%s.log'%(DIR,NAME),'w',0)
        with pyutil.RedirectStdStreams(logFile):
            mdl.fixMean= fixMean
            mdl.fit(X)
#             reorderByMSQ(mdl)
            if reorder:
                mdl.reorderByMSQ()
            d.update({'suc':1,'model':mdl})
#             logFile.close()
        print "[SUCC] to fit Model:%s"%(NAME,)
        print qcmsg.msgGMM(mdl)
    except Exception as e:
        print "[FAIL] to fit Model:%s due to :'%s'"%(NAME,e)
        d.update({'suc':0})
    if model_only:
        d['train_data'] = None
        d['rowName'] = None
        d['colName'] = None
    np.save('%s/%s'%(DIR.rstrip('/'),NAME),d)
    d = scount.countMatrix.from_dict(d)
    return d


Overwriting modelRoutine.py


In [105]:
import pymisca.vis_util as pyvis
import pymisca.util as pyutil
plt = pyvis.plt
import synotil.modelRoutine as smod
import synotil.CountMatrix as scount
import synotil.qcplots as sqc
import synotil.dio as sdio
import synotil.norm as snorm


is in ipython: 1 
[WARN]No module named jinja2_util


[WARN] pymisca.vis_util cannot find network
[WARN] pymisca.vis_util cannot find network


is in ipython: 1 
[WARN]No module named jinja2_util


In [133]:
import pymisca.tensorflow_extra_.hyper_plane_mixture as hpm
# hpm.main._fit_MAP??

In [2]:
%%writefile jobs.py
# import sy
# import scipy.cluster.hierarchy as sphier
# import scipy.spatial.distance as spdist
import pymisca.vis_util as pyvis
import pymisca.util as pyutil
plt = pyvis.plt; pd = pyutil.pd; np = pyutil.np
import pymisca.models as pymod
import pymisca.ext as pyext

import synotil.modelRoutine as smod
import synotil.CountMatrix as scount
import synotil.PanelPlot as spanel
import synotil.qcplots as sqc
import synotil.dio as sdio
job__nearAUG = sdio.job__nearAUG

import synotil.norm as snorm
sjob= pyutil.sys.modules[__name__]


def figs__peakBW(peakFile,
                 bwFiles,
                 outerRadius = 500,
                 innerRadius=50, 
                 stepSize=10,
                 center_summit = 0,
                 outIndexFunc=None,
                 outIndex=pyutil.basename,
                 ylim = [0,None],
                 NCORE=4,
                 squareSize=(0.2,0.01),
                 detailByChip = None,
                 detailByGene = 0, ### runtime linear in number of genes
                 name = None,
                 **kwargs):
    ### legacy
    if outIndexFunc is not None:
        assert outIndex is None
        outIndex= outIndexFunc
    else:
        assert outIndex is not None
        pass
    #### get the data ready
    bwTracks = sdio.extract_bigwig_multiple(bedFile=peakFile,fnames=bwFiles,
                                            radius=outerRadius,NCORE=NCORE,
                                             stepSize=stepSize,
                                             outIndex=outIndex,
                                            center_summit = center_summit,
                                            **kwargs)
    if detailByChip is None:
        if len(bwTracks) <= 100:
            detailByChip = 1
        else:
            detailByChip = 0
            
    if name is None:
        name = pyutil.getBname(peakFile)
        
    poss = bwTracks.columns.levels[1]
    innerPos = poss[abs(poss) <= innerRadius]

    bwAvg = pyutil.colGroupMean(bwTracks.reindex(columns=innerPos,level=1))
    bwAvg = scount.countMatrix(bwAvg).apply(pyutil.log2p1)

    ##### plotting
#     fig,axs = plt.subplots(1,2,figsize=[12,6])
#     figs = {}
    figs = pyutil.collections.OrderedDict()
    #########
#     plt.figure(figsize=[6,4])
    fig,axs = plt.subplots(1,3,figsize=[18,6])

    ax =axs[0]
    bwAvg.boxplot(rot='vertical',ax=ax)
    ax.set_ylabel('log2(peak intensity)')
    ax.set_ylim(ylim)

    ax =axs[1]
    sqc.qc_pileUp(bwTracks,ax=ax,axLeg=axs[2])
    figs['pileUp-%s' % name ]  = plt.gcf()
    ax.set_ylim(ylim)
    
    #########
    bwAvg.heatmap(figsize=[20,10])
    figs['avgHeatmap-%s' % name] =plt.gcf()
    
    pos = bwTracks.columns.get_level_values('pos')
    cname = 'binding'
    #########
    if detailByChip:
        for key,bwTrack in bwTracks.groupby(axis=1,level=0):
            dfc = bwTrack
            pyvis.heatmap(dfc,
                          transpose=0,
                          squareSize=(0.025,0.2),
                          ytick = dfc.index,
                          xlab = 'distance to %s' % key,
                         vlim=ylim,
                         cname=cname)
            
            ax = plt.gca()
            xticks = pos[ax.get_xticks().astype(int)[:-1]]
            ax.set_xticklabels(xticks,)

#             pyvis.heatmap(bwTrack,
#                           transpose=1,
#                           squareSize=squareSize,
#                           xtick = bwTracks.index,ylab = key)
            figs['detailByChip-%s/%s'%(name,key)] = plt.gcf()
    
    ########
    if detailByGene:

        for key,bwTrack in bwTracks.groupby(axis=0,level=0):
            dfc = bwTrack.melt().pivot_table(index='bwFile',
                                                 columns='pos',)
            pyvis.heatmap(dfc,
                          transpose=0,
                          squareSize=(0.025,0.2),
                          ytick = dfc.index,
#                           xtick = dfc.columns,                          
                          xlab = 'distance to %s' % key,
                          vlim=ylim,
                         cname = cname)
            ax = plt.gca()
            xticks = pos[ax.get_xticks().astype(int)[:-1]]
            ax.set_xticklabels(xticks,)
            
            figs['detailByGene-%s/%s'%(name,key)] = plt.gcf()
            


    
    
    return figs,(bwTracks,bwAvg)

panel_kw_dft =  dict(
    figsize=[14, 7],
    show_axa = 1,
    show_axc = 0,
    showGrid = 0,
    width_ratios = [1,14,0.]
#     title= '',
#     height_ratios = [1,3,3,3,1],
    )
def job__render__panelPlot(
    tracks = None,
    clu=None,
    order=None,
    index=None,
    aliasFmt = '{alias}',
    alias = None,
    baseFile=0,
    figsize = None,
    panel_kw = panel_kw_dft,
    how = 'left',
    debug=0,
    extra = {},
    **kwargs
):
    if figsize is not None:
        panel_kw['figsize'] = figsize
    autoAli = alias is None
    if autoAli:
        alias = ''
    if isinstance(clu,basestring):
        alias += pyext.getBname(clu)
        clu = pyutil.readData(clu,baseFile=baseFile)
    if isinstance(order,basestring):
        alias += pyext.getBname(order)
        order = pyutil.readData(order,baseFile=baseFile)
    if isinstance(tracks,basestring):
        alias += pyext.getBname(tracks)
        tracks = pyutil.readData(tracks,baseFile=baseFile)
        tracks = list(tracks)
    if isinstance(panel_kw,basestring):
        alias += pyext.getBname(panel_kw)
        panel_kw = pyutil.read__buffer( panel_kw,
                                       ext='json',typ='rec',
                                       guess_index=0).to_dict()
    if order is not None:
        clu = order.get(['clu'])
    else:
        assert clu is not None
        order = pd.DataFrame(clu)
        
    if isinstance(index,basestring):
        alias += pyutil.sanitise_query(index)
        locals().update(extra)
        index = eval(index)

    cluTrack = spanel.fixCluster(clu.get(['clu']))
    alias = aliasFmt.format(**locals())
#     cluFile_clean = 'clean_%s.csv' % alias
#     cluc.to_csv(cluFile_clean)    
    tracks = pyext.list__realise(tracks, locals())
    ##### Output heatmap
    pp = spanel.panelPlot(tracks,**panel_kw)
    pp.compile(how=how,
               index=index,
               **kwargs
              )
    pp.compile(order=order)
#     if debug:
#         return pp
    if debug:
        return pp
    fig = pp.render();    
    return (alias,fig)

def job__rawFile__combine(dfc,silent=0):
    '''take a dataframe that identifies raw files and
    convert it to combined fastq.gz ready for downstream.
'''
    dfc = sdio.rawFile__validateChunk(dfc)
    dfcCombined = dfc.groupby('fnameCombined',
                              as_index=False
                             ).apply(sdio.rawFile__combineChunk,
                                            silent=0)
    return dfcCombined


def worker__md5sum(row,column='fname'):
    checkSum_col = '%s_md5sum' % column
    dct= row
    val = dct[column]
    res = pyutil.getMD5sum(val)
    dct[checkSum_col] = res
    return dct

def job__md5sum(dfc,column='fname',n_cpu=1, **kwargs):
    worker = pyutil.functools.partial(worker__md5sum,
                                     column=column)
    it = (x.__dict__ for x in dfc.itertuples())
    res = pyutil.mp_map(worker, it ,n_cpu = n_cpu, **kwargs)
#     res = pd.DataFrame(res)
    return res

def fig__fluffProfile(interval,tracks,ofname = None,
                      annotation= None,
                      scaleOpt = None, 
                      fragmentSize=0,
                      labels = None,
                        silent = 0):
#     annotation = BED12
    # cmd = 'fluff profile'
    trackFlat = u' '.join(tracks)
    if scaleOpt is None:
        scaleOpt = ' -s 1:%d ' % (len(tracks))
    if ofname is None:
        ofname = interval + '.svg'
    cmd =  ''
    cmd += ' fluff profile '
    cmd +=  scaleOpt
    cmd += ' -f {fragmentSize} '
    if annotation is not None:
        cmd += ' -a {annotation} '
    if labels is not None:
        labelFlat = u' '.join(labels)
        cmd += ' -l {labelFlat}'
    cmd += ' -o {ofname} -i {interval} -d {trackFlat} '
    cmd += ' -b white '
    cmd += ' 2>&1 '
    cmd = cmd.format(**locals())
    res=  pyutil.shellexec(cmd,silent=silent)
    return ofname

fig__fluffyProfile = fig__fluffProfile ###legacy


def job__cluster__hpm(tdf,name='test0', 
                      K =40 ,meanNorm=1,
                      threshold=0.,batchSize=500,n_iter=3000,
                      silent=0,
                      NCORE=4,
                      randomState=0,
                      alpha = None,
                      weighted = True,
                     ):
    import pymisca.tensorflow_extra_.hyper_plane_mixture as hpm
    hpm.tf.set_random_seed(randomState)
    np.random.seed(randomState)
    mdl =  hpm.main(K=K,NCORE=NCORE,name=name,meanNorm=meanNorm,threshold=threshold,
                   weighted=weighted,alpha = alpha)
    if batchSize==0 or batchSize is None:
        batchMaker = None
#         batchMaker = hpm.pytfu.batchMaker__random(batchSize=batchSize)
    else:
         batchMaker = hpm.pytfu.batchMaker__random(batchSize=batchSize)
    res = mdl.fit(tdf,
            batchMaker = batchMaker,
            n_iter=n_iter,autoStop=0,
    )
    if not silent:
#         import matplotlib.pyplot as plt
        plt.plot(res)
    cdict = pymod.cache__model4data(mdl,tdf) 
#     assert 0
    mdl.post.__dict__.update(cdict)
    np.save('params.npy', mdl.params)
    res = mdl.params
    res['mdl'] = mdl
    return pyutil.util_obj(**res)

def job__cluster__vmf(tdf,
                      K = 30, init_method='kmeans',
                      weighted= True,
                      n_iter=3000,randomState=None,
                    nStart = 15,
                      min_iters = 50,
                    verbose=1,
                    callback= None,
                    silent=0,
                      sample_weights= 'sd',
):
    import pymisca.model_collection.mixture_vmf as mod    
    np.random.seed(randomState)
    mdl = mod.MixtureVMF(K=K,init_method=init_method,
                         weighted = weighted,
                        )

    histLoss = mdl.fit(tdf,
                       verbose=verbose,
                       callback=callback,
                       nStart=nStart,
                       n_iter = n_iter,
                       min_iters = min_iters,
                       sample_weights =sample_weights,
                      )
    histLoss = -histLoss
    
    if not silent:
#         import matplotlib.pyplot as plt
        plt.plot(histLoss)    
    cdict = pymod.cache__model4data(mdl,tdf) 
    cdict.update(mdl.params)

    np.save('params.npy', cdict)

    cdict['mdl'] = mdl
    return pyutil.util_obj(**cdict)

def job__combinePeak(bwCurr,
                     featSummit='/home/feng/ref/Arabidopsis_thaliana_TAIR10/annotation/genes.gtf.cds.summit',
                     GSIZE='/home/feng/ref/Arabidopsis_thaliana_TAIR10/genome.sizes',
                    CUTOFF=4000,
                     head = 1000,
                    alias = 'testPeaks',
                    center_summit = 1,):

    bwCurr = bwCurr.dropna(subset=['npkFile'])
    bwCurr['npkFileLine'] = bwCurr.eval("npkFile.map(@pyutil.lineCount)")
    print (bwCurr[['bname','npkFileLine']].sort_values('bname'))

    dfs = zip(bwCurr.index,map(sdio.extract_peak,bwCurr.npkFile))
    dfs = dict(dfs)

    dfs = {k:v.sort_values('score',ascending=False) for k,v in dfs.items()}
    bed = pd.concat([df.head(head) for df in dfs.values()],axis=0,sort=False)
    bed = bed.dropna(axis=1)

    ofname = pyutil.to_tsv( bed,'%s__combined.bed' % alias)
    if (len(dfs) > 1) and center_summit :
        ofname = sdio.bed__merge(ofname,silent=0)
    # pyutil.print
    print(ofname,pyutil.lineCount(ofname))

    bedFile= ofname
    
#     peakSummit = sdio.bed__summit(bedFile)
    peakSummit = sdio.npk_expandSummit(fname=bedFile,radius=1,
                                       center_summit=center_summit)
    peak2geneFile = sdio.job__nearAUG(featSummit=featSummit,
                                       peakSummit=peakSummit,
                                       GSIZE=GSIZE,peakWid=1,
                                       CUTOFF=CUTOFF)

    pyutil.fileDict__save('files.json',
                          d=locals(),
                          keys= ['bedFile','peakSummit','peak2geneFile'])
    return 'files.json'


def job__chipTargPaired(
    bwCurr  = None,
    bwMeta = None, control= None, treatment = None,
    xlab = None, ylab = None,
    name = None,
#     bwMeta,
    NCORE = 2,
    params__peakBW = None,
    CUTOFF_FC = 3.0,
    CUTOFF_CHIPDIFF = 0.7,
    innerRadius = 100,
):
    figs = pyutil.collections.OrderedDict()

    
    if control is not None and treatment is not None:
        xlab,ylab = control,treatment
    if xlab is None or ylab is None:
        xlab,ylab = bwCurr.index
    elif bwCurr is None:
        bwCurr = bwMeta.reindex([xlab,ylab])
        
    if params__peakBW is None:
        
        params__peakBW = dict(
            outerRadius=500,
            innerRadius=innerRadius,
            NCORE = NCORE,
            outIndex=bwCurr.header,
        #     detailByCHIP = 0,
        )
    params__peakBW['innerRadius'] = innerRadius        
        
    if name is None:
        name = '{xlab}-{ylab}'.format(**locals())
#     bwCurr = bwMeta
#     bwCurr = bwCurr.loc[[xlab,ylab]]

#     bwCurr.npkFile

    dfs = map(sdio.extract_peak, bwCurr.npkFile,)
    
    fig,ax = plt.subplots(1,1,figsize=[7,7])
#     ax = plt.gca()
    for df in dfs:
        df['per_FC']  = pyutil.dist2ppf(df.FC)
        df.plot.scatter('per_FC','FC',ax=ax)

    fnames = [ pyutil.queryCopy(infile=fname,
                                query='FC>%.3f'%CUTOFF_FC ,
                                reader=sdio.extract_peak,inplace=False) 
        for fname in bwCurr.npkFile]
    # dfs[1]

    peakFlat = ' '.join(fnames)
    ofname = '%s-combined.bed' % ('-'.join(bwCurr.index))
    pyutil.shellexec('cat {peakFlat}>{ofname}'.format(**locals()))
    ofname = sdio.npk_expandSummit(fname=ofname, radius=1)

    pyutil.lineCount(ofname)
    peakFileOrig = peakFile = ofname


    res = sjob.figs__peakBW(
        peakFile = peakFile,
        bwFiles = bwCurr.RPKMFile,
        name = name,
        **params__peakBW
    )
    figs.update(res[0])

    bwTrack,bwAvg = res[1]
    bwAvg.columns = bwCurr.index

    xs,ys = bwAvg[[xlab,ylab]].values.T
#     clu = None
    query = ' val_{ylab} - val_{xlab} > {CUTOFF_CHIPDIFF} '.format(**locals())
    qsans = pyutil.sanitise_query(query)
    
#     peakIndex = pyutil.df__pad(bwAvg).query(query).index
    clu = pd.DataFrame(pyutil.df__pad(bwAvg))
    peakIndex = clu.query(query).index
    clu['clu']= clu.eval('index in @peakIndex')
    
    pyvis.qc_2var(xs,ys,clu=clu.clu,xlab=xlab,ylab=ylab)
    figs['scatterPlot__%s' % name ]= plt.gcf()
    cluFile = ofname = qsans+'.csv'
    clu.to_csv(ofname)
    print (ofname,pyutil.lineCount(ofname))
    peakBase = pyutil.getBname(peakFile)
    ofname =  '{peakBase}-{qsans}.bed'.format(**locals())
    peakFile = pyutil.to_tsv(
        sdio.extract_peak(peakFile).set_index('acc',drop=0).reindex(peakIndex),
                           ofname)
    pyutil.shellexec('mkdir -p output/')
    pyutil.file__link(ofname,'output/%s.bed' % name,force=True)
    
#     peakFile = pyutil.queryCopy(peakFile,
#                                 query='acc in @peakIndex',
#                                 reader=sdio.extract_peak, 
#                                 peakIndex=peakIndex,
#                                )
#     peakFile =  '{peakFile}-{qsans}.bed'
#     pyutil.fileDict__main(ofname='FILE.json',
#                          **pyutil.dictFilter(locals(),
#                                              keys=['cluFile','peakFile',
#                                             'peakFileOrig']
#                                             ))
    
    pyutil.fileDict__save(d=locals(), keys=['cluFile','peakFile',
                                            'peakFileOrig'],fname='FILE.json')
    return figs,clu

Overwriting jobs.py


In [13]:
sjob.figs__peakBW??

In [4]:
??pyutil.fileDict__save


In [ ]:
de

In [2]:
execfile('/home/feng/meta/header_0903.py')

[WARN] NUMPY is not limited cuz NCORE is not set
[WARN] NUMPY is not limited cuz NCORE is not set
[WARN] pymisca.vis_util cannot find network


is in ipython: 1 
[WARN]No module named jinja2_util


[WARN] pymisca.vis_util cannot find network
[WARN] Cannot find file:key.gene
[WARN] Cannot find file:key.gene
[WARN] pymisca.vis_util cannot find network
[WARN] pymisca.vis_util cannot find network


is in ipython: 1 
[WARN]No module named jinja2_util
is in ipython: 1 
[WARN]No module named jinja2_util
is in ipython: 1 
[WARN]No module named jinja2_util
/home/feng/.local/lib/python2.7/site-packages/pandas/core/computation/check.py:17: UserWarning:

The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6




In [1]:
! grep -rnw *ipynb *py -e "def shot2html"

dev.ipynb:1806:    "def shot2html(indexFile):\n",
shot2html.py:4:def shot2html(indexFile):


In [ ]:
%%writefile /home/feng/repos/BrachyPhoton/synotil/shot2html.py
import synotil.dio as sdio
# import PIL
# import cairosvg
# import pymisca.util as pyutil
import os
def make_thumbnail(path, size):
#     newpath = os.path.dirname(path)
#     image_name = os.path.basename(os.path.splitext(path)[0])
#     newpath = newpath + '/' + image_name + '.thumbnail.jpg'


    try:
        sp = path.rsplit('.',1)
        if len(sp)==1:
            ext = ''
            base = sp[0]
        else:
            base,ext = sp
        if ext.lower()=='svg':
            base = path.rsplit('.',1)[0]
            with open(path,'r') as f:
                svg = f.read()
            cairosvg.svg2png(bytestring=svg,write_to=base + '.png')
            path = base + '.png'
            ext = 'png'

        newpath =  base + '__thumbnail.' + ext 
        img = PIL.Image.open(path)
        img.thumbnail(size)
        img.save(newpath, None)

    except Exception as e:
        newpath = path
        print('[WARN]image error for PATH %s:\n%s'%(newpath,e))
    return newpath


def shot2html(indexFile,localPath = True):
# indexFile = "PROG=chipShots_bedFile=64-SD-ZT20-ELF3OX-RERUN_S8_peaks.narrowPeak.FC-GT-3dot0/index.tsv"
    dfc = sdio.extract_peak(indexFile)
    dfc['img'] = dfc[dfc.columns[-1]]
#     THUMBNAIL_SIZE = 256, 256
    # HTML_REPORT_NAME='test.html'
    ofname = HTML_REPORT_NAME='%s.html'%indexFile 
    TITLE = os.path.basename(indexFile)
    # !pip install --user cairosvg==1.0.22
    # cairosvg.svg2png?
    # from cairosvg import svg2png

    # generate an html page

    fhtml = open(HTML_REPORT_NAME, 'w')
#     TITLE = 'test'
    fhtml.write('''
    <html>
    <head>
    <style> 
        body {{font-family:\"HelveticaNeue-Light\", \"Helvetica Neue Light\", \"Helvetica neue\"}}
    </style>
    <title>IGV screenshots for {TITLE}</title>
    '''.format(**locals()))

    js = """
    <script>
    //var frame = window.frames["postFrame"];
    //var frame = document.getElementsByName("postFrame")[0];

    //dir = document.URL.replace(/^.*[\\\/]/, '')
    //var frame = window.open("about:blank","postFrame", 'height=440,width=260,scrollbars=yes');
    console.log(document.URL);


    // Pass the checkbox name to the function
    function checkbox() {
       // var frame = window.frames["postFrame"];
    //    var doc = frame.document;
    //  var doc = frame.contentDocument || frame.contentWindow.document

      //opened.open();
      var checkboxes = document.getElementsByName("line");
      var checkboxesChecked = [];

      // loop over them all
      html_string = '';
      for (var i=0; i<checkboxes.length; i++) {
         // And stick the checked ones onto an array...
         if (checkboxes[i].checked) {
            checkboxesChecked.push(checkboxes[i]);
            html_string += checkboxes[i].value + '<br>';
    //        doc.writeln(checkboxes[i].value);
    //        doc.write("<br>");
         }
      }
      src = "data:text/html;charset=utf-8," + escape(html_string);
       var frame = window.open(src,"postFrame", 'height=440,width=260,scrollbars=yes');

      //opened.close()
      // Return the array if it is non-empty, or null
      return checkboxesChecked.length > 0 ? checkboxesChecked : null;
    }
    </script>
    </head>

    """

    fhtml.write(js)



    fhtml.write('<body>\n')

    # fc = []
    # for t in image_lst:
    #     narrowpeak_line = t[0]
    #     lst = narrowpeak_line.split()
    #     fc.append(float(lst[4]))

    # max_fc = max(fc)
    # min_fc = min(fc)
    # lmd = 0.8
    # show_color_val = (1.0 - lmd) * min_fc +  1.0 * lmd * max_fc


    colors = ['#ff9999','#ff8080','#ff6666','#ff4d4d','#ff3333','#ff1a1a','#ff0000']
    sz_color = len(colors)

    fhtml.write('<p><b>%s</b></p>' % ('&nbsp; &nbsp;'.join(['Peak_position','Peak_name', 'Integer_score_for_display', 'Dot','fold-change',' -log10pvalue',' -log10qvalue','Relative_summit_position_to_peak_start']))) # head

    fhtml.write('<iframe name="postFrame"></iframe>')
    fhtml.write('<form>')
    fhtml.write('<input type=\"submit\" value=\"Save\" size=\"35\" onClick=\"return checkbox();\">\n')

    for i in range(len(dfc)):
        row = dfc.iloc[i]
    #     if np.isnan(row.img):

        if isinstance(row.img,float):
            imgFile = './none'
        else:
            imgFile = row.img
            
        if localPath:
            imgFile = './%s'% os.path.basename(imgFile)
        else:
            imgFile = '/notebooks%s' % imgFile            
            
        bedLine = ('\t').join(map(str,row[:-1]))
    #     bedLine = '\t'.join(row)
    #     line = line_lst[i]
        ind = i
        fhtml.write('<input type=\"checkbox\" id=\"%s\" name=\"line\" value=\"%s\" >' % (ind, 
                                                                                         bedLine.replace('\t','\t')
                                                                                        ))
        c = colors[0]
        fhtml.write('<font color=%s>%s</font><br/>\n' % (c,bedLine))
        fhtml.write('<font color=%s>%s</font><br/>\n' % (c, bedLine.replace('\t','&emsp; ')))
    #     timgFile = make_thumbnail(imgFile, THUMBNAIL_SIZE)
    #     if JUPYTER:

    
    #     timgFile = '/notebooks' + timgFile
    #     if imgFile.endswith('.svg'):
    #         imgFile = imgFile.replace('.svg','.png')

        fhtml.write('<a href=\"%s\">\n<img src=\"%s\"></a><br/><br/>\n' % (imgFile, imgFile))

    fhtml.write('</form>')    
    fhtml.write('</body>\n')
    fhtml.write('</html>\n')
    fhtml.close()    

    # os.remove(PYTHON_SCRIPT_NAME)

    print('[Done.]')
    return ofname



In [8]:
! ls /home/feng/Templates/listImages.html

/home/feng/Templates/listImages.html


In [70]:
import json
??json.dumps({},sort_keys=True)